In [100]:
import numpy as np
from bs4 import BeautifulSoup
import pymongo
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError
import urllib3
import json
import multiprocessing
import time

In [20]:
client = MongoClient()
db = client['dota']

In [37]:
pro_id_coll = db.pro_matches_id
pro_matches_full_coll = db.pro_matches_full

In [123]:
def get_working_proxy():
    user_agent = {'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; Win64; x64;en; rv:5.0) \
                  Gecko/20110619 Firefox/5.0'}
    proxy_list_site = 'https://free-proxy-list.net/'
    pm = urllib3.PoolManager(1, 
                             headers=user_agent, 
                             cert_reqs='CERT_REQUIRED')
    prx_req = pm.request('GET', proxy_list_site)
    prx_soup = BeautifulSoup(prx_req.data, 'html.parser')
    proxies_list = []
    for tr in prx_soup.find_all('tr'):
        tmp = []
        for td in tr.find_all('td')[:2]:
            tmp.append(td.text)
        if len(tmp) < 2:
            continue
        proxies_list.append([str(tmp[0]) + ':' + tmp[1]])
    
    proxies_list = proxies_list[:20]
    np.random.seed()
    np.random.shuffle(proxies_list)
    for i, prx in enumerate(proxies_list):
        try:
            print('Getting working proxy...')
            cur_prx_address = 'https://' + str(prx[0]) + '/'
            prx_http = urllib3.ProxyManager(cur_prx_address,
                                            maxsize=1, 
                                            headers=user_agent, 
                                            cert_reqs='CERT_REQUIRED')
            r = prx_http.request('GET', 'https://stackoverflow.com/', timeout=0.7)
            if r.status == 200:
                print('Proxy found.', cur_prx_address)
                return cur_prx_address
            time.sleep(0.1)
        except Exception as err:
            # print(err)
            continue

In [124]:
def api_get_match(prx_address, match_id):
    url = 'https://api.opendota.com/api/matches/'
    user_agent = {'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; Win64; x64;en; rv:5.0) \
                  Gecko/20110619 Firefox/5.0'}
    cur_url = url + str(match_id)
    prx_m = urllib3.ProxyManager(prx_address, headers=user_agent, cert_reqs='CERT_REQUIRED')
    r = prx_m.request('GET', cur_url, timeout=2)
    matches = json.loads(r.data)
    return matches

In [125]:
def get_new_ids():
    ids_all = [x['match_id'] for x in pro_id_coll.find({}, {'match_id'})]
    ids_stored = [x['match_id'] for x in pro_matches_full_coll.find({}, {'match_id'})]
    need_to_load_ids_bool = np.isin(ids_all, ids_stored, invert=True)
    need_to_load_ids = np.array(ids_all)[need_to_load_ids_bool]
    return ids_all

In [126]:
def load_insert(matches_ids, odd_even):
    user_agent = {'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; Win64; x64;en; rv:5.0) \
                  Gecko/20110619 Firefox/5.0'}
    prx_address = get_working_proxy()
    pm = urllib3.ProxyManager(proxy_url=prx_address, headers=user_agent, cert_reqs='CERT_REQUIRED')
    
    for i, cur_match_id in enumerate(matches_ids):
        print(i, cur_match_id) if i % 10 == 0 else None
        if cur_match_id % 2 == odd_even:
            try:
                cur_match = api_get_match(prx_address, cur_match_id)
                if hasattr(cur_match, 'error'):
                    prx_address = get_working_proxy()
                    continue
                pro_matches_full_coll.insert_one(cur_match)
            except Exception as e:
                print(e)
                continue
        time.sleep(0.1)

In [ ]:
needed_ids = get_new_ids()
print(len(needed_ids))

pool1 = multiprocessing.Pool(2)
pool2 = multiprocessing.Pool(2)
p1 = pool1.Process(target=load_insert, args=(needed_ids, 0, ))
p2 = pool2.Process(target=load_insert, args=(needed_ids, 1, ))
p1.start()
p2.start()
p1.join()
p2.join()

55929
Getting working proxy...
Getting working proxy...
Getting working proxy...
Getting working proxy...
Getting working proxy...
Getting working proxy...
Proxy found. https://138.68.16.228:8080/
0 3944966018
Getting working proxy...


/home/nemo/anaconda3/lib/python3.6/site-packages/pymongo/topology.py:149: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "


E11000 duplicate key error collection: dota.pro_matches_full index: match_id_1 dup key: { : 3944966018 }
Getting working proxy...
Proxy found. https://163.172.86.64:3128/
0 3944966018
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
E11000 duplicate key error collection: dota.pro_matches_full index: match_id_1 dup key: { : 3944647926 }
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
10 3944054871
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
E11000 duplicate key error collection: dota.pro_matches_full index: match_id_1 dup key: { : 3944463864 }
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
20 3943365614
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1

/home/nemo/anaconda3/lib/python3.6/site-packages/pymongo/topology.py:149: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "


E11000 duplicate key error collection: dota.pro_matches_full index: match_id_1 dup key: { : 3942041515 }
Expecting value: line 1 column 1 (char 0)
E11000 duplicate key error collection: dota.pro_matches_full index: match_id_1 dup key: { : 3943547236 }
20 3943365614
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
50 3941885974
Expecting value: line 1 column 1 (char 0)
E11000 duplicate key error collection: dota.pro_matches_full index: match_id_1 dup key: { : 3943365614 }
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
E11000 duplicate key error collection: dota.pro_matches_full index: match_id_1 dup key: { : 3941612813 }
Expecting value: line 1 column 1 (char 0)
60 3941422211
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
E11000 duplicate key error collection: dota.pro_matches_full index: match_id_1 dup key: { : 3942922478 }
Expecting value: line 1 column 1 (char 0)
Expecting val